# PSET 1 

### Juan M Jimenez

## Q2.

In [1]:
#Pkg.add.(["Plots", "Statistics", "Distributions", "LinearAlgebra", "PrettyTables", "Random"])
#Pkg.add.("Random")
#Pkg.add("PrettyTables")

In [2]:
cd(joinpath(pwd(),".."))

using Pkg
Pkg.activate(".") ;

  Activating project at `C:\Github\UBC-phd`


In [3]:
#Pkg.add("PrettyTables")
using Plots, Statistics, Distributions, LinearAlgebra, PrettyTables, Random

## a.
Generate n = 500 observations for ($Y_i$;$X_i$;$Z_i$) from the following IV regression model with heteroskedastic errors.

In [4]:
#Seed for randomness
Random.seed!(1234)

TaskLocalRNG()

In [5]:
# Defining parameters 
β=1
Π=[1; 1]
ρ=0.95
μ=[0;0]
Σ=[1 ρ; ρ 1]

function sample_data(n)
    
    #Joint distribution of (ϵ,v)
    jointDist = MvNormal(μ, Σ)
    errors=rand(jointDist,n)'
    
    ϵ=errors[:,1]
    v=errors[:,2]
    Z=randn(n,2)
    
    #Models
    X=Z*Π+v
    U=exp.(Z*Π) .* ϵ
    Y=X*β+U
    
    #Outputs
    return (Y = Y , X = X , Z = Z)
end

sample_data (generic function with 1 method)

In [6]:
Ys , Xs, Zs = sample_data(1000);

## b. 
Compute $\beta^{2SLS}$ and its standard error.

In [7]:
function est2sls(Y,X,Z)
       
    #Estimator
    PZ = Z*( (Z'*Z)\Z' )
    β2sls = (X'*PZ*X)\(X'*PZ*Y)
    
    #Calculating Ω
    U= Y - X*β
    n=length(U)
    #Ω= (U'*U*Z'*Z)./n    <------------ ASK!!!!!!!!!!!!!
    zr = Z.*U
    Ω = (zr' * zr)/n
    
    #Standard error
    Q= (Z'*X)./n
    W= inv((Z'*Z)./n)
    σ2_2sls= ( (Q'*W*Q)\(Q'*W*Ω*W*Q)/(Q'*W*Q) )./n
    σ_2sls= sqrt(σ2_2sls)
    
    return β2sls, σ_2sls
end

est2sls (generic function with 1 method)

In [8]:
beta, std = est2sls(Ys,Xs,Zs)

(1.1475087824316017, 0.23769452702587562)

## c.
Compute $\beta^{GMM}$, the two-step GMM effcient estimator of $\beta$, and its standard error.


In [9]:
function estgmm(Y,X,Z)

    # Initial GMM estimator using W=A'A=I
    #W0 = [1 0;0 1]
    W0 = I
    Q= (Z'*X)
    β0_gmm = (Q'*W0*Q)\(Q'*W0*Z'Y)
    
    # Two-step standard error
    U= Y - β0_gmm*X
    n=length(U)
    #Ω2= (U2'*U2*Z'*Z)./n
    zr = Z.*U
    Ω = (zr' * zr)/n
    
    W=inv(Ω)
    σ2_gmm=inv((Q'*W*Q)/n)
    σ_gmm=sqrt(σ2_gmm)
       
    #Final estimator
    βgmm = (Q'*W*Q)\(Q'*W*Z'Y)
    
    return βgmm, σ_gmm
end

estgmm (generic function with 1 method)

In [10]:
beta, std = estgmm(Ys,Xs,Zs)

(1.1178460888458592, 0.2320372531206108)

In [11]:
#Setting vars to zero
ns=10000

tbias_2sls=0
tbias_gmm=0

tσ_2sls=0
tσ_gmm=0

tinci_2sls=0
tinci_gmm=0

#Using a normal distribution for testing sice n is large (otherwise t dist)
zstat= quantile(Normal(0,1), .975)

1.9599639845400576

## d.
Generate 10,000 independent samples of size n from the model. For each sample
compute the absolute bias and check whether $\beta$ is inside the estimator's 95% CIs.

In [12]:
for s=1:ns
    #Creating data
    Ys , Xs, Zs = sample_data(500)
    
    #Estimating coefficients
    beta_2sls, std_2sls = est2sls(Ys,Xs,Zs)
    beta_gmm, std_gmm = estgmm(Ys,Xs,Zs)
    
    #Calculating total biases
    tbias_2sls += abs(beta_2sls-β)
    tbias_gmm += abs(beta_gmm-β)
    tσ_2sls += std_2sls
    tσ_gmm += std_gmm
    tinci_2sls += (abs(β-beta_2sls)<=zstat*std_2sls)
    tinci_gmm += (abs(β-beta_gmm)<=zstat*std_gmm)   
    
    # Other way for CI:
    #tinci_2sls += (β>beta_2sls - zstat*std_2sls)*(β<beta_2sls + zstat*std_2sls)
    #tinci_gmm += (β>beta_gmm - zstat*std_gmm)*(β<beta_gmm + zstat*std_gmm)
end

## e. 
Report the average bias, the average standard error, and the simulated coverage for the 95% CIs.

In [13]:
#Calculating mean biases and coverages
mbias_2sls=tbias_2sls/ns
mbias_gmm=tbias_gmm/ns
mσ_2sls=tσ_2sls/ns
mσ_gmm=tσ_gmm/ns
minci_2sls=tinci_2sls/ns
minci_gmm=tinci_gmm/ns;

In [14]:
#Table report
table_data = ["Bias" mbias_2sls mbias_gmm mbias_2sls-mbias_gmm;
        "Ave. std.err" mσ_2sls mσ_gmm mσ_2sls-mσ_gmm;
        "Coverage Prob of CI" minci_2sls minci_gmm minci_2sls-minci_gmm;]
header=["Statistic", "2SLS", "Two-step efficient GMM", "Difference"]
pretty_table(table_data; header)

┌─────────────────────┬──────────┬────────────────────────┬────────────┐
│           Statistic │     2SLS │ Two-step efficient GMM │ Difference │
├─────────────────────┼──────────┼────────────────────────┼────────────┤
│                Bias │ 0.450479 │               0.367889 │  0.0825905 │
│        Ave. std.err │ 0.535834 │               0.449416 │  0.0864177 │
│ Coverage Prob of CI │   0.9677 │                 0.9543 │     0.0134 │
└─────────────────────┴──────────┴────────────────────────┴────────────┘


## f.
Compare the two methods (2SLS and two-step efficient GMM) in terms of the
statistics listed in the previous part. Which is the preferred method?

Given N=500, I would choose the GMM method since the GMM estimator has a less average bias and a less average standard error too. Which means not only that the GMM estimates are less biased than the 2SLS ones, but also that they are more precise.  

## g.
Repeat (d)-(f) for n = 100 and discuss the differences with the simulation results for n = 500.

In [15]:
#Setting vars to zero
ns=10000

tbias_2sls=0
tbias_gmm=0
tσ_2sls=0
tσ_gmm=0
tinci_2sls=0
tinci_gmm=0

#z-statistic
zstat= quantile(Normal(0,1), .975)

#Siulation
for s=1:ns
    #Creating data
    Ys , Xs, Zs = sample_data(100)
    
    #Estimating coefficients
    beta_2sls, std_2sls = est2sls(Ys,Xs,Zs)
    beta_gmm, std_gmm = estgmm(Ys,Xs,Zs)
    
    #Calculating total biases
    tbias_2sls += abs(beta_2sls-β)
    tbias_gmm += abs(beta_gmm-β)
    tσ_2sls += std_2sls
    tσ_gmm += std_gmm
    tinci_2sls += (abs(β-beta_2sls)<=zstat*std_2sls)
    tinci_gmm += (abs(β-beta_gmm)<=zstat*std_gmm)
    
    #Calculating mean biases and coverages
    mbias_2sls=tbias_2sls/ns
    mbias_gmm=tbias_gmm/ns
    mσ_2sls=tσ_2sls/ns
    mσ_gmm=tσ_gmm/ns
    minci_2sls=tinci_2sls/ns
    minci_gmm=tinci_gmm/ns
end

In [16]:
#Table report
table_data = ["Bias" mbias_2sls mbias_gmm mbias_2sls-mbias_gmm;
        "Ave. std.err" mσ_2sls mσ_gmm mσ_2sls-mσ_gmm;
        "Coverage Prob of CI" minci_2sls minci_gmm minci_2sls-minci_gmm;]
header=["Statistic", "2SLS", "Two-step efficient GMM", "Difference"]
pretty_table(table_data; header)

┌─────────────────────┬──────────┬────────────────────────┬────────────┐
│           Statistic │     2SLS │ Two-step efficient GMM │ Difference │
├─────────────────────┼──────────┼────────────────────────┼────────────┤
│                Bias │ 0.819678 │                0.62935 │   0.190328 │
│        Ave. std.err │ 0.929378 │               0.716174 │   0.213204 │
│ Coverage Prob of CI │   0.9766 │                 0.9362 │     0.0404 │
└─────────────────────┴──────────┴────────────────────────┴────────────┘


Given N=100, I would choose the GMM method since the GMM estimator has a less average bias and a less average standard error too. Which means not only that the GMM estimates are less biased than the 2SLS ones, but also that they are more precise.  

Furthermore, it seems the difference in the magnitud of the biases has increased in a smaller sample, which leads me to believe that GMM is even more accurate in samller samples. 